# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
import pandas as pd
import sqlite3


In [2]:
conn=sqlite3.connect("data.sqlite")

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:
query=(""" SELECT
customerNumber,
contactLastName,
contactFirstName
FROM customers
WHERE customerNumber IN (
SELECT customerNumber
FROM orders
WHERE orderDate = '2003-01-31'
);""",conn)

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
query = """
SELECT
customerNumber,
contactLastName,
contactFirstName
FROM
customers
WHERE
customerNumber IN (
SELECT customerNumber
FROM orders
WHERE orderDate = '2003-01-31'
)
"""


results = pd.read_sql_query(query, conn)

print(results)


   customerNumber contactLastName contactFirstName
0             141          Freyre           Diego 


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [5]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

query = """
SELECT
products.productName,
COUNT(DISTINCT orders.customerNumber) AS numCustomers
FROM
products
JOIN
orderdetails ON products.productCode = orderdetails.productCode
JOIN
orders ON orderdetails.orderNumber = orders.orderNumber
GROUP BY
products.productName
ORDER BY
numCustomers DESC
"""

results = pd.read_sql_query(query, conn)

print(results)

                             productName  numCustomers
0            1992 Ferrari 360 Spider red            40
1                       Boeing X-32A JSF            27
2                    1972 Alfa Romeo GTA            27
3               1952 Alpine Renault 1300            27
4                     1934 Ford V8 Coupe            27
..                                   ...           ...
104  1958 Chevy Corvette Limited Edition            19
105                  2002 Chevy Corvette            18
106            1969 Chevrolet Camaro Z28            18
107                    1952 Citroen-15CV            18
108                   1949 Jaguar XK 120            18

[109 rows x 2 columns]


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [7]:
import pandas as pd

# Step 1: Find the product codes for products that have been ordered by fewer than 20 people
subquery_1 = """
    SELECT DISTINCT productCode
    FROM orderdetails
    GROUP BY productCode
    HAVING COUNT(DISTINCT orderNumber) < 20
"""
product_codes = pd.read_sql_query(subquery_1, conn)

if len(product_codes) > 0:
    # Step 2: Find the order numbers associated with the product codes from Step 1
    subquery_2 = """
        SELECT DISTINCT orderNumber
        FROM orderdetails
        WHERE productCode IN ({})
    """.format(", ".join(map(str, product_codes['productCode'])))
    order_numbers = pd.read_sql_query(subquery_2, conn)

    if len(order_numbers) > 0:
        # Step 3: Find the employee numbers associated with the orders from Step 2
        subquery_3 = """
            SELECT DISTINCT employeeNumber
            FROM orders
            WHERE orderNumber IN ({})
        """.format(", ".join(map(str, order_numbers['orderNumber'])))
        employee_numbers = pd.read_sql_query(subquery_3, conn)

        if len(employee_numbers) > 0:
            # Step 4: Retrieve the employee details for the employee numbers from Step 3
            main_query = """
                SELECT
                    e.employeeNumber,
                    e.firstName,
                    e.lastName,
                    o.city,
                    o.officeCode
                FROM
                    employees e
                JOIN
                    offices o ON e.officeCode = o.officeCode
                WHERE
                    e.employeeNumber IN ({})
            """.format(", ".join(map(str, employee_numbers['employeeNumber'])))
            result = pd.read_sql_query(main_query, conn)

            # Display the result
            print(result)
        else:
            print("No employees found.")
    else:
        print("No order numbers found.")
else:
    print("No product codes found.")


No product codes found.


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [8]:

# Query to retrieve the employee details and count of customers
query = """
    SELECT
        e.employeeNumber,
        e.firstName,
        e.lastName,
        COUNT(c.customerNumber) AS numberOfCustomers
    FROM
        employees e
    JOIN
        customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    GROUP BY
        e.employeeNumber,
        e.firstName,
        e.lastName
    HAVING
        AVG(c.creditLimit) > 15000
"""

# Execute the query and fetch the result
result = pd.read_sql_query(query, conn)

# Display the result
print(result)


    employeeNumber firstName   lastName  numberOfCustomers
0             1165    Leslie   Jennings                  6
1             1166    Leslie   Thompson                  6
2             1188     Julie   Firrelli                  6
3             1216     Steve  Patterson                  6
4             1286  Foon Yue      Tseng                  7
5             1323    George     Vanauf                  8
6             1337      Loui     Bondur                  6
7             1370    Gerard  Hernandez                  7
8             1401    Pamela   Castillo                 10
9             1501     Larry       Bott                  8
10            1504     Barry      Jones                  9
11            1611      Andy     Fixter                  5
12            1612     Peter      Marsh                  5
13            1621      Mami      Nishi                  5
14            1702    Martin     Gerard                  6


In [9]:
conn.commit()

In [10]:
conn.close()

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!